In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [ ]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        itmes = text.strip().split(",")
        return list(map(str.strip, itmes))

['Hello', 'how', 'are', 'you']

In [41]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else",
        ),
        ("human", "{question}"),
    ]
)

In [ ]:
chain = template | chat | CommaOutputParser()
chain.invoke({"max_itmes": 5, "question": "What are the pokemons?"})